## The Movies Dataset TF-IDF Content-Based Recommendation
### Сырцев Александр, M34031, 2022

# Задание 1

In [1]:
import pandas as pd
import numpy as np

загружаем файл movies_metadata

In [2]:
metadata_dtypes = {'id': 'int64'}
metadata = pd.read_csv('movies_metadata.csv', sep=',', dtype=metadata_dtypes, low_memory=False)

In [3]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45463 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

удалим все фильмы с пустым overview

In [4]:
shaped_metadata = metadata
selection = shaped_metadata[(shaped_metadata['overview'] == 'No overview found.') | (shaped_metadata['overview'] == 'No Overview') | (shaped_metadata['overview'] == ' ')]
shaped_metadata = shaped_metadata.drop(selection.index, axis=0)
shaped_metadata['overview'].describe()

count                                                 44364
unique                                                44303
top       King Lear, old and tired, divides his kingdom ...
freq                                                      3
Name: overview, dtype: object

# Задание 2
оставим только колонки 'id', 'imdb_id', 'overview', 'title'

In [5]:
shaped_metadata2 = shaped_metadata.drop(['adult', 'belongs_to_collection', 'budget', 'homepage', 'original_language', 'original_title', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'video', 'vote_average', 'vote_count', 'genres'], axis=1)
shaped_metadata2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45318 entries, 0 to 45462
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45318 non-null  int64 
 1   imdb_id   45301 non-null  object
 2   overview  44364 non-null  object
 3   title     45318 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


Выведем случайные 10 строк

In [6]:
samples = shaped_metadata2.sample(n=10)
print(samples)

           id    imdb_id                                           overview  \
22920   22259  tt1473345  Once the ruler of an entire universe, the Grea...   
42987  128284  tt0020321  A beautiful and evocative portrait of Ivens' n...   
38529   40713  tt1194664  During her freshman year in high school, Mika ...   
27019   95004  tt0021931  An acclaimed actor and his equally acclaimed a...   
34278  100783  tt2290845  Yedyanchi Jatra is an enjoyable portrayal of h...   
16505   57448  tt1606382  A documentary on the relationship between fash...   
18692  119364  tt0072249  A policeman in New Mexico takes payoff money b...   
5595    10863  tt0082351  A ruthless German spy, trying to get out of Br...   
7933    36627  tt0072021  Following With Fire and Sword', this is an ada...   
23275  210479  tt2692904  Ivan Locke has worked hard to craft a good lif...   

                             title  
22920  Bionicle: The Legend Reborn  
42987                         Rain  
38529              

# Задание 3

загружаем данные, убеждаемся в отсутсвии пропусков

In [7]:
ratings = pd.read_csv("ratings.csv")
ratings.head(10)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [8]:
pd.isnull(ratings).sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Склеим 2 датасета

In [9]:
superset = pd.merge(shaped_metadata2, ratings, left_on='id', right_on='movieId')
superset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11402375 entries, 0 to 11402374
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   imdb_id    object 
 2   overview   object 
 3   title      object 
 4   userId     int64  
 5   movieId    int64  
 6   rating     float64
 7   timestamp  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 782.9+ MB


# Задание 4

Посчитаем пропуски в получившемся датасете

In [30]:
pd.isnull(superset).sum()

id               0
imdb_id         47
overview     40657
title            0
userId           0
movieId          0
rating           0
timestamp        0
dtype: int64

Они существуют, посмотрим на наличие пропусков в датасете из задания 2

In [32]:
pd.isnull(shaped_metadata2).sum()

id            0
imdb_id      17
overview    954
title         0
dtype: int64

Получается, что пропуски появились потому что они были в датасете из второго задания. Удалим их:

In [35]:
clean_superset = superset.dropna()
pd.isnull(clean_superset).sum()

id           0
imdb_id      0
overview     0
title        0
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Выведем размеры получившегося датасета

In [36]:
clean_superset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11361671 entries, 0 to 11402374
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   imdb_id    object 
 2   overview   object 
 3   title      object 
 4   userId     int64  
 5   movieId    int64  
 6   rating     float64
 7   timestamp  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 780.1+ MB


И 10 случайных записей

In [37]:
superset_samples = clean_superset.sample(n=10)
print(superset_samples)

              id    imdb_id  \
5549120     2470  tt0103791   
5650970      296  tt0181852   
4145404      318  tt0120753   
10231572  107916  tt2177509   
8221056       58  tt0383574   
9973911    25855  tt1551621   
1840369     4995  tt0118749   
8888920    81847  tt0020815   
1640355      364  tt0103776   
11057247  104374  tt0060787   

                                                   overview  \
5549120   In 1931, a young soldier deserts from the army...   
5650970   It's been 10 years since John Connor saved Ear...   
4145404   The Million Dollar Hotel starts with a jump fr...   
10231572  Carmina, 58, runs a shop selling Iberian produ...   
8221056   Captain Jack Sparrow works his way out of a bl...   
9973911   A violent gang is abducting and killing women ...   
1840369   Set in 1977, back when sex was safe, pleasure ...   
8888920   World War I ace Dick Courtney derides the lead...   
1640355   Having defeated the Joker, Batman now faces th...   
11057247  A married woman le